In [ ]:
import torchgen
import numpy as np
from PIL import Image
from pathlib import Path
import random
from termcolor import colored

from LUT.models_x import *
from LUT import torchvision_x_functional as TF_x
import torchvision.transforms.functional as TF

In [ ]:
folder_path = Path('demo/')
lut_path = folder_path / 'lut.txt'
img_path = folder_path / 'img.jpg'
img_output_path = folder_path / 'output_lut.jpg'

assert folder_path.is_dir and folder_path.exists
assert lut_path.is_file and lut_path.exists
assert img_path.is_file and img_path.exists

In [ ]:
check_preprocessing = True
if check_preprocessing:
    id = random.randint(0, 349)
    id = 271
    lut_file_name = '%04d_0.txt' % id
    lut_path = 'testspace/lut/' + lut_file_name
    print(f'Select LUT {lut_file_name} to process image.')

In [ ]:
# create LUT object on CUDA
assert torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor
trilinear_ = TrilinearInterpolation()

LUT0 = Generator3DLUT_zero(dim=0)
dim = 33

file = open(str(lut_path), 'r')
lines = file.readlines()
buffer = np.zeros((3,dim,dim,dim), dtype=np.float32)

for i in range(0,dim):
    for j in range(0,dim):
        for k in range(0,dim):
            n = i * dim*dim + j * dim + k
            x = lines[n].split()
            buffer[0,i,j,k] = float(x[0])
            buffer[1,i,j,k] = float(x[1])
            buffer[2,i,j,k] = float(x[2])
LUT0.LUT = nn.Parameter(torch.from_numpy(buffer).requires_grad_(True))
LUT0 = LUT0.cuda()
LUT = LUT0.LUT

# read image and transform to tensor
img = Image.open(str(img_path))
img = TF.to_tensor(img).type(Tensor)
img = img.unsqueeze(0)

In [ ]:
# generate image
_, result = trilinear_(LUT, img)

# save image
ndarr = result.squeeze().mul_(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
im = Image.fromarray(ndarr)
im.save(str(img_output_path), quality=95)